In [1]:
import matplotlib.pyplot as plt
import numpy as np

#%connect_info
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms

from torchvision.datasets import MNIST

# MNIST con Pytorch
Pytorch facilita la creación de redes neuronales, como ejemplo puedes tomar la siguiente red que clasifica a los datos del conjunto MNIST haciendo uso de la arquitectura vista anteriormente: una capa de entrada con 784 neuronas, 27 neuronas en una capa oculta y una salida de 10 unidades.

Para implementar una red con esta herramienta debes extender a la clase nn.Module y a sus métodos:
- Constructor: donde estableces cuáles serán las capas que componen a la arquitectura
- Forward: donde indicas como se conectan las capas indicadas en el constructor

Ademas, para simplificar el proceso puedes agregar un metodo **train** al cual le pases el numero de iteraciones y otros parametros para entrenar la red.

In [2]:
#Descargamos el conjunto de datos en las variables.
train_dataset_mnist = MNIST(root='./data',train=True,download=True,
                          transform=transforms.ToTensor())
test_dataset_mnist  = MNIST(root='./data',train=False,download=True,
                          transform=transforms.ToTensor())

params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}

#Creamos un DataLoader,el cual nos permite iterar sobre nuestros datos y 
#administrarlos facilmente en lotes.
train_loader_mnist  = torch.utils.data.DataLoader(dataset=train_dataset_mnist,batch_size=64,
                                                 shuffle=True)

test_loader_mnist   = torch.utils.data.DataLoader(dataset=test_dataset_mnist,batch_size=64,
                                                 shuffle=True)

In [3]:
def reshape_mnist(batch):
    '''Recibe un batch de informacion de MNIST y lo aplana
    Ej: [5,28,28]-> [5,784]'''
    return batch.view(-1,28*28)

def contarCorrectas(net,batch,labels,func=None):
    '''Dado un batch y sus etiquetas, cuenta el numero de respuestas
    correctas de una red, el parametro func aplica una modificacion al 
    tensor que contiene los datos'''
    
    if(func!=None):
        batch=func(batch)
        salidas=net(batch)
    else:
        salidas=net(batch)
    respuestas=salidas.max(dim=1)[1]
    cantidadCorrectas=(respuestas==labels).sum()
    return cantidadCorrectas
    
def calcularPrecisionGlobal(net,data_loader,batch_size,func=None,cuda=False):
    '''Calcula la precision de una red dado un data_loader,
    recive una funcion que transforma los datos en caso de ser necesario'''
    correctas=0
    for (images,labels) in data_loader:
        if(cuda and torch.cuda.is_available()):
            images=images.cuda()
            labels=labels.cuda()
        correctas+=contarCorrectas(net,images,labels,func)        
    correctas=correctas.data.tolist()
    return (100*correctas)/(len(data_loader)*batch_size)

In [4]:
#Nuestro modelo debe heredar de nn.Module
class MNIST_NET(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(MNIST_NET,self).__init__()
        #Definimos nuestras unidades de entrada y las agregamos al objeto actual
        self.fc1 = nn.Linear(input_size,hidden_size,bias=True)
        self.sig1= nn.Sigmoid() 
        self.fc2 = nn.Linear(hidden_size,num_classes,bias=True)        
    
    def forward(self,input_data):
        '''Definimos como seran procesados los datos de entrada'''
        out=self.fc1(input_data) #Primero se hacen pasar por la capa lineal
        out=self.sig1(out)       #Despues,la salida para por la funcion sigmode
        out=self.fc2(out)        #La salida finalmente pasa por otra capa lineal
        return out

    def train(self,epochs,data_loader,criterion,optimizer,cuda=False):
        '''Entrena la red net, por un numero de epocas "epochs" con el data_loader
        proporcionado,usando como funcion de perdida la definida en "criterion" y el
        optimizador pasado como parametro'''
        for epoch in range(epochs):
            for i,(images,labels) in enumerate(data_loader):

                images = Variable(images.view(-1,28*28))#aplanamos el batch actual
                labels = Variable(labels.view(-1,))     #convertimos el tensor a un tensor de valores unitarios
                if(cuda and torch.cuda.is_available()): #Si nuestra PC cuenta con GPU,realizamos calculos en ella
                    images=images.cuda()
                    labels=labels.cuda()

                optimizer.zero_grad()
                outputs= self(images)#llamar a nuestro objeto con parametros es introducirlos a la red
                loss   = criterion(outputs,labels)
                loss.backward()
                optimizer.step()

            if (epoch) % 2 == 0:                              # Logging
                print('Epoch [%d/%d], Step %d, Loss: %.4f'
                 %(epoch, epochs, i+1, loss.item()))
            
#Instanciamos la red que ya conocemos para MNIST de 28*28 --> 27 --> 10
Red =MNIST_NET(28*28,27,10)
#Red.cuda() #puedes descomentar esta linea si tienes GPU disponible
#Escogemos la funcion de error para el modelo (entropia cruzada)
criterio   = nn.CrossEntropyLoss()
optimizer  = torch.optim.SGD(Red.parameters(),lr=0.001)
#Entrenamos la red durante 100 pasos,con entropia cruzada y SGD 
Red.train(25,train_loader_mnist,criterio,optimizer,cuda=False) #puedes agregar cuda=true si tienes GPU disponible

Epoch [0/25], Step 938, Loss: 2.2803
Epoch [2/25], Step 938, Loss: 2.2437
Epoch [4/25], Step 938, Loss: 2.1891
Epoch [6/25], Step 938, Loss: 2.1272
Epoch [8/25], Step 938, Loss: 2.0490
Epoch [10/25], Step 938, Loss: 2.0188
Epoch [12/25], Step 938, Loss: 1.9636
Epoch [14/25], Step 938, Loss: 1.8086
Epoch [16/25], Step 938, Loss: 1.7294
Epoch [18/25], Step 938, Loss: 1.7050
Epoch [20/25], Step 938, Loss: 1.4583
Epoch [22/25], Step 938, Loss: 1.3667
Epoch [24/25], Step 938, Loss: 1.4056


In [5]:
prec_train =calcularPrecisionGlobal(Red,train_loader_mnist,64,func=reshape_mnist,cuda=False)
prec_val   =calcularPrecisionGlobal(Red,test_loader_mnist,64,func=reshape_mnist,cuda=False)
print("Precision en conjunto de entrenamiento: %.4f%%"%(prec_train))
print("Precision en conjunto de validacion: %.4f%%"%(prec_val))

Precision en conjunto de entrenamiento: 70.3875%
Precision en conjunto de validacion: 71.1485%
